In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

# Create a synthetic dataset
data = {
    'user': ['fatimah', 'aeem', 'zuni', 'bilal', 'hana'],
    'Movie1': [5, 4, np.nan, 2, np.nan],
    'Movie2': [4, np.nan, 4, 1, 2],
    'Movie3': [np.nan, 3, 5, 1, 3],
    'Movie4': [2, 2, 1, 5, 4],
    'Movie5': [1, 1, np.nan, 4, 5]
}
ratings = pd.DataFrame(data)

# Fill missing values with 0
user_item_matrix = ratings.set_index('user').fillna(0)

item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

# Train Nearest Neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_item_matrix.T)

def recommend_movies(user_name, top_n=3):
    """
    Recommend movies for a given user based on item-based collaborative filtering.

    Args:
    - user_name: The name of the user for whom recommendations are needed.
    - top_n: Number of movies to recommend.

    Returns:
    - A DataFrame of recommended movies with predicted ratings.
    """
    if user_name not in user_item_matrix.index:
        return f"User '{user_name}' not found in the dataset."

    user_ratings = user_item_matrix.loc[user_name]
    unrated_movies = user_ratings[user_ratings == 0].index

    predicted_ratings = {}

    for movie_id in unrated_movies:
        similar_movies = item_similarity_df[movie_id].sort_values(ascending=False).drop(movie_id).head(10)
        weighted_sum = 0
        similarity_sum = 0

        for similar_movie, similarity_score in similar_movies.items():
            if user_ratings[similar_movie] > 0:  # Only consider rated movies
                weighted_sum += similarity_score * user_ratings[similar_movie]
                similarity_sum += similarity_score

        predicted_ratings[movie_id] = weighted_sum / similarity_sum if similarity_sum > 0 else 0

    recommendations = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return pd.DataFrame(recommendations, columns=['movie_id', 'predicted_rating'])

# User input
user_name = input("Enter your username: ")
top_n = 3  # Number of recommendations
recommendations = recommend_movies(user_name, top_n)
print(recommendations)
